In [26]:
import pandas as pd
import os
import json
import xlrd
import datetime
import locale
import numpy as np

In [299]:
# data Nilai perlu diubah menjadi numerik secara manual di excel.
# diubah dari contohnya 100,00 menjadi 100.00
master_data = pd.read_excel('seleksi mitra/25032023_Seleksi Kompertensi dan Wawancara Online.xlsx', converters={'NIK':str})
ujian_kompetensi = pd.read_excel('seleksi mitra/24032023 ST2023_seleksi_Jambi-UJIAN SELEKSI CALON PETUGAS ST2023-jawaban.xlsx', converters={'Nilai/100':float, 'Jawaban 5':str})
wawancara_online = pd.read_excel('seleksi mitra/24032023 ST2023_seleksi_Jambi-WAWANCARA (online)-jawaban.xlsx', converters={'Nilai/100.00':float, 'Jawaban 6':str})

In [300]:
# rename column
ukMapper = {
    "Jawaban 5":"NIK",
    "Alamat Email":"Email",
    "Jawaban 4": "Nama",
    "Jawaban 6": "Telp" ,
    
}
wwMapper = {
    "Jawaban 6": "NIK",
    "Alamat Email":"Email",
    "Jawaban 5": "Nama",
    "Jawaban 7": "Telp" ,
    "Nilai/100.00":'Nilai/100'

}

ujian_kompetensi.rename(columns=ukMapper, inplace=True)
wawancara_online.rename(columns=wwMapper, inplace=True)


In [301]:
ujian_kompetensi['Email'] = [x.lower() for x in ujian_kompetensi['Email']]
wawancara_online['Email'] = [x.lower() for x in wawancara_online['Email']]
ujian_kompetensi['Nama'] = [x.lower() for x in ujian_kompetensi['Nama']]
wawancara_online['Nama'] = [x.lower() for x in wawancara_online['Nama']]
master_data['NAMA'] = [x.lower() for x in master_data['NAMA']]

In [302]:
master_data.loc[0]

NIK                                    1506020709030000
KODE CALON PETUGAS                                 SK_1
NIK CENSORED                           1506020******004
NAMA                                ahmad khoirul anwar
form pendaftaran                              terdaftar
sobat                                         terdaftar
No Telepon                                 082281374048
KECAMATAN                                 SEBERANG KOTA
Email                         babankanwar2003@gmail.com
Email di SOBAT BPS            *****kanwar2003@gmail.com
HASIL SELEKSI ADMINISTRASI                        Lulus
Name: 0, dtype: object

In [303]:
def getHighestValue(df, colNames):
    highest = -1
    indexRes = 0
    for index, row in df.iterrows():
        if row[colNames]>highest:
                highest = row[colNames]
                indexRes = index
    return highest, indexRes

def processDf(df:pd.DataFrame, row):
     # cek dari NIK
     
     ujianKompRow = df.loc[df['NIK']==row['NIK']]
     if len(ujianKompRow)>0:
          highest, indexRes = getHighestValue(ujianKompRow, 'Nilai/100')
          # print(df.iloc[indexRes]['Email'], resRow.iloc[indexRes]['Keadaan'], highest)
          return df.iloc[indexRes]['Email'], df.iloc[indexRes]['Keadaan'], highest
     
     # cek dari email 
     ujianKompRow = df.loc[df['Email']== row['Email']]
     if len(ujianKompRow)>0:
          highest, indexRes = getHighestValue(ujianKompRow, 'Nilai/100')
          return df.iloc[indexRes]['Email'], df.iloc[indexRes]['Keadaan'], highest
     
     # cek dari nama
     ujianKompRow = df.loc[df['Nama']== row['NAMA']]
     if len(ujianKompRow)>0:
          highest, indexRes = getHighestValue(ujianKompRow, 'Nilai/100')
          return df.iloc[indexRes]['Email'], df.iloc[indexRes]['Keadaan'], highest
     
     # cek dari no telepon
     ujianKompRow = df.loc[df['Telp']== row['No Telepon']]
     if len(ujianKompRow)>0:
          highest, indexRes = getHighestValue(ujianKompRow, 'Nilai/100')
          return df.iloc[indexRes]['Email'], df.iloc[indexRes]['Keadaan'], highest
     
     # kalo ngga ada yang match
     return float('NaN'), float('NaN'), float('NaN')

def selecAndProcessDf(dataName, row):
     match dataName:
          case 'kompetensi':
               email, keadaan, highest = processDf(ujian_kompetensi, row)
               return email, keadaan, highest
          case "wawancara":
               email, keadaan, highest = processDf(wawancara_online, row)
               return email, keadaan, highest
          case _:
               print('no data')


In [304]:
temp = ujian_kompetensi.loc[ujian_kompetensi['NIK']=='1506034103000002']
if len(temp)>0:
    val1, val2 = getHighestValue(temp, 'Nilai/100')
    print(val1, val2)

263.0 235


In [305]:
ujian_kompetensi.loc[ujian_kompetensi['NIK']=='1506034103000002']

,Nama akhir,Nama Depan,Email,Keadaan,Dimulai pada,Selesai,Waktu mengerjakan,Nilai/100,Jawaban 1,Jawaban 2,...,Jawaban 7,Jawaban 8,Jawaban 9,Jawaban 10,Jawaban 11,Jawaban 12,Jawaban 13,Jawaban 14,Jawaban 15,Jawaban 16
235,Anita,Nur,nuranita1357@gmail.com,Selesai,22 Maret 2023 17:19,22 Maret 2023 17:34,14 min 48 detik,263.0,nuranita1357@gmail.com,Pengabuan,...,Ekspor-Impor,Tebu,Menteri Perencanaan Pembangunan Nasional,Nama sesuai KTP,Inflasi,Babi Hutan,7,Indeks Pembangunan Manusia,Undang-Undang (UU) No. 16 Tahun 1997 tentang S...,2020


In [307]:
finalDf = master_data.copy()
# finalDf['Email Wawancara'] = np.nan
# finalDf['Status Wawancara'] = np.nan
# finalDf['Nilai/100 Wawancara'] = np.nan
# finalDf['Email Kompetensi'] = np.nan
# finalDf['Status Kompetensi'] = np.nan
# finalDf['Nilai/100 Kompetensi'] = np.nan

finalRes = []
for index, row in finalDf.iterrows():
    res = {}
    res['NIK'] = row['NIK']
    res['KODE CALON PETUGAS'] = row['KODE CALON PETUGAS']
    res['NIK CENSORED'] = row['NIK CENSORED']
    res['NAMA'] = row['NAMA']
    res['form pendaftaran'] = row['form pendaftaran']
    res['sobat'] = row['sobat']
    res['No Telepon'] = row['No Telepon']
    res['KECAMATAN'] = row['KECAMATAN']
    res['Email'] = row['Email']
    res['Email di SOBAT BPS'] = row['Email di SOBAT BPS']
    res['HASIL SELEKSI ADMINISTRASI'] = row['HASIL SELEKSI ADMINISTRASI']
    
    email, keadaan, highest = selecAndProcessDf('wawancara', row)
    res['Email Wawancara'] = email
    res['Status Wawancara'] = keadaan
    res['Nilai/100 Wawancara Tertinggi'] = highest

    email, keadaan, highest = selecAndProcessDf('kompetensi', row)
    res['Email Kompetensi'] = email
    res['Status Kompetensi'] = keadaan
    res['Nilai/100 Kompetensi Tertinggi'] = highest

    finalRes.append(res)
        
        
        

KeyError: 'Nama'

In [ ]:
master_data.iloc[335]['NIK']

1506034103000002

In [ ]:
finalRes[1]

{'NIK': 1506022602900001,
 'KODE CALON PETUGAS': 'SK_2',
 'NIK CENSORED': '1506022******001',
 'NAMA': 'dodi herwanto',
 'form pendaftaran': 'terdaftar',
 'sobat': 'terdaftar',
 'No Telepon': '0082377699043',
 'KECAMATAN': 'SEBERANG KOTA',
 'Email': 'Doddy.mutan@gmail.com',
 'Email di SOBAT BPS': '*****.mutan@gmail.com',
 'HASIL SELEKSI ADMINISTRASI': 'Lulus',
 'Email Wawancara': nan,
 'Status Wawancara': nan,
 'Nilai/100 Wawancara Tertinggi': nan,
 'Email Kompetensi': nan,
 'Status Kompetensi': nan,
 'Nilai/100 Kompetensi Tertinggi': nan}

In [ ]:
jsonObj = json.dumps(finalRes)
finalObjPd = pd.read_json(jsonObj)
finalObjPd
finalObjPd.to_csv('hasil/25032023 hasil seleksi petugas ST.csv', index=False)

In [ ]:
finalObjPd

,NIK,KODE CALON PETUGAS,NIK CENSORED,NAMA,form pendaftaran,sobat,No Telepon,KECAMATAN,Email,Email di SOBAT BPS,HASIL SELEKSI ADMINISTRASI,Email Wawancara,Status Wawancara,Nilai/100 Wawancara Tertinggi,Email Kompetensi,Status Kompetensi,Nilai/100 Kompetensi Tertinggi
0,1506020709030000,SK_1,1506020******004,ahmad khoirul anwar,terdaftar,terdaftar,082281374048,SEBERANG KOTA,babankanwar2003@gmail.com,*****kanwar2003@gmail.com,Lulus,babankanwar2003@gmail.com,Selesai,1000.0,babankanwar2003@gmail.com,Selesai,368.0
1,1506022602900001,SK_2,1506022******001,dodi herwanto,terdaftar,terdaftar,0082377699043,SEBERANG KOTA,Doddy.mutan@gmail.com,*****.mutan@gmail.com,Lulus,None,None,NaN,None,None,NaN
2,1506105208010001,SK_3,1506105******001,eni purwati,terdaftar,terdaftar,082281238019,SEBERANG KOTA,enipurwati6780@gmail.com,*****rwati6780@gmail.com,Lulus,enipurwati6780@gmail.com,Selesai,850.0,enipurwati6780@gmail.com,Selesai,263.0
3,1506024302960001,SK_4,1506024******001,fahru raziah se,terdaftar,terdaftar,082374008210,SEBERANG KOTA,zzia1643@gmail.com,*****643@gmail.com,Lulus,zzia1643@gmail.com,Selesai,780.0,zzia1643@gmail.com,Selesai,263.0
4,1506021010970003,SK_5,1506021******003,fahrul lazi s.h,terdaftar,terdaftar,082377847748,SEBERANG KOTA,fahrullazi1997@gmail.com,*****llazi1997@gmail.com,Lulus,fahrullazi1997@gmail.com,Selesai,925.0,fahrullazi1997@gmail.com,Selesai,263.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
516,1506070806820000,BTA_42,1506070******000,suyono,None,terdaftar,None,BATANG ASAM,suyonoyono47982@gmail.com,*****oyono47982@gmail.com,Lulus,None,None,NaN,None,None,NaN
517,1506076207040000,BTA_43,1506076******000,tara tista,None,terdaftar,None,BATANG ASAM,tistatara34@gmail.com,*****tara34@gmail.com,Lulus,None,None,NaN,None,None,NaN
518,1506070107020006,BTA_44,1506070******006,waldi lucio parera,terdaftar,terdaftar,None,BATANG ASAM,parerawaldilucio@gmail.com,*****awaldilucio@gmail.com,Lulus,parerawaldilucio@gmail.com,Selesai,1000.0,parerawaldilucio@gmail.com,Selesai,316.0
519,1506011707950000,BTA_45,1506011******000,yendi m,None,terdaftar,None,BATANG ASAM,Yendimardi@gmail.com,*****mardi@gmail.com,Lulus,None,None,NaN,None,None,NaN
